### Add model, connection and device to settings:

In [43]:
# библиотеки
import os
import sys
import inspect
import numpy as np
import re
import math
import matplotlib.pyplot as plt
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from hybriddomain.envs.hs.model.model_main import ModelNet as Model
from hybriddomain.settings.settings_main import Settings
from ddeint import ddeint

# пути и названия файлов
modelFileName = 'problems/Diplom_Maslenikov'
conn_name = "connection.json"
device_conf_name = "devices.json"

#Ввод пароля для инициализации пользователя
model = Model()
model.io.loadFromFile(modelFileName)
settings = Settings(model, conn_name, device_conf_name, conn_name)

INFO:settings_main:device_conf_rpath
INFO:settings_main:devices.json


Please enter password for user imaslenikov:


 ·········


In [44]:
print("equations:")
print(model.equations)
print("params:")
print(model.params)
print("paramValues:")
print(model.paramValues)

equations:
[["U'= V", "V'= (b*sin(V(t-1))-V-k*e*U)/e"]]
params:
['e', 'k', 'b']
paramValues:
[{'e': 0.01, 'k': 2, 'b': 0.1}, {'e': 0.01, 'k': 2, 'b': 0.9}]


### Run solver:

In [45]:
# запуск процесса решения уравнения
from hybriddomain.solvers.hs.remoterun.progresses.progress_notebook import ProgressNotebook
from hybriddomain.solvers.hs.postproc.results.results_main import ResultPostprocNet as ResultPostproc
progress = ProgressNotebook(100, prefix='solving')

progress_params = ProgressNotebook(len(model.paramValues), prefix='param')
# display(progress_params.progress)

progress_solving = ProgressNotebook(100, prefix='solving')
# display(progress_solving.progress)
# progress_solving.succ(0)

result_postproc = ResultPostproc(modelFileName)

from hybriddomain.solvers.hs.remoterun.remoterun import remoteProjectRun

clear_output()
test_name = settings.paths['model']['name']
bTest = widgets.Button(description="run %s " % (test_name))

# parametrisation:
def run(event):
    model = Model()
    model.io.loadFromFile(modelFileName)
    
    for params_idx in range(len(model.paramValues)):
        
        progress_params.succ(params_idx)
        progress_solving.set_prefix("solving")
        
        # changing param:
        model.defaultParamsIndex = params_idx
        model.io.saveToFile(settings.paths["hd"]["json"])
        model = Model()
        model.io.loadFromFile(modelFileName)
        
        remoteProjectRun(settings, 1, progress_solving,remove_old=False)
        
        # result_postproc.extract_out(params_idx, results_params)
        result_postproc.rename_out(params_idx)
    display(Javascript("alert('Done');"))
    
bTest.on_click(run)

display(progress_params.progress)
display(progress_solving.progress)
 
display(bTest)

IntProgress(value=0, description='param: ', max=1)

IntProgress(value=0, description='solving: ', max=99)

Button(description='run Diplom_Maslenikov ', style=ButtonStyle())

### Results:

In [46]:
model.readResults(result_format=1)
# available result files:
print("result files: ", model.results_paths)
# available plot files:
print("plot files: ", model.plots_paths)

result files:  {'u': ['problems/Diplom_Maslenikov/out/u_seq0.out', 'problems/Diplom_Maslenikov/out/u_seq1.out']}
plot files:  {}


In [47]:
# объекте model хранится вся информация по решению задачи
# get results (for TRAC-11 result_format=1):
model.readResults(result_format=1)
model.results_paths
# available results:
print(model.results_arrays.keys())

dict_keys(['u'])


### Video:

In [48]:
# available plots:
model.plots_paths

{}

In [49]:
display(HTML(model.get_video('System',0)))
display(HTML(model.get_video('System',1)))
display(HTML(model.get_video('System',2)))
display(HTML(model.get_video('System',3)))
display(HTML(model.get_video('System',4)))

KeyError: 'System'

### U(x, t-fixed),V(x, t-fixed):

In [ ]:
# отрисовывает результат
%matplotlib inline

#display(w)
time = model.results_arrays['u']['timevalues']
print(time)

times = model.results_arrays['u']['timevalues']
#print(times)
result = model.results_arrays['u']['resvalues']
print(result)
#print(len(times))
result_x = np.array([result[0][time][0][0] for time in times])
#result_y = np.array([result[0][time][1][0] for time in times])
#plt.plot(time, result_x)

#tt =np.linspace (0.0,1.0,101)
#circle_x=np.array([10*math.cos(2*3.1415926*t) for t in tt])
#circle_y=np.array([10*math.sin(2*3.1415926*t) for t in tt])


def f(x,t):
    result_x = np.array([result[x][time][0][t] for time in times])
    result_y = np.array([result[x][time][1][t] for time in times])
#    # Сама траектория зависимость от t берется разное начало запуска
    plt.plot(time, result_x)
#    # Круг на котором лежат начала траекторий 
#    plt.plot(circle_x, circle_y)
w = interactive(f,x=(0,len(result)-1,1),t=(0,3,1))
display(w)
#print(len(result))
plt.show()

[0.0, 0.0101, 0.0201, 0.0301, 0.0401, 0.0501, 0.0601, 0.0701, 0.0801, 0.0901, 0.1001, 0.1101, 0.1201, 0.1301, 0.1401, 0.1501, 0.1601, 0.1701, 0.1801, 0.1901, 0.2001, 0.2101, 0.2201, 0.2301, 0.2401, 0.2501, 0.2601, 0.2701, 0.2801, 0.2901, 0.3001, 0.3101, 0.3201, 0.3301, 0.3401, 0.3501, 0.3601, 0.3701, 0.3801, 0.3901, 0.4001, 0.4101, 0.4201, 0.4301, 0.4401, 0.4501, 0.4601, 0.4701, 0.4801, 0.4901, 0.5001, 0.5101, 0.5201, 0.5301, 0.5401, 0.5501, 0.5601, 0.5701, 0.5801, 0.5901, 0.6001, 0.6101, 0.6201, 0.6301, 0.6401, 0.6501, 0.6601, 0.6701, 0.6801, 0.6901, 0.7001, 0.7101, 0.7201, 0.7301, 0.7401, 0.7501, 0.7601, 0.7701, 0.7801, 0.7901, 0.8001, 0.8101, 0.8201, 0.8301, 0.8401, 0.8501, 0.8601, 0.8701, 0.8801, 0.8901, 0.9001, 0.9101, 0.9201, 0.9301, 0.9401, 0.9501, 0.9601, 0.9701, 0.9801, 0.9901, 1.0001, 1.0101, 1.0201, 1.0301, 1.0401, 1.0501, 1.0601, 1.0701, 1.0801, 1.0901, 1.1001, 1.1101, 1.1201, 1.1301, 1.1401, 1.1501, 1.1601, 1.1701, 1.1801, 1.1901, 1.2001, 1.2101, 1.2201, 1.2301, 1.2401, 1.